In [1]:
import numpy as np
from matplotlib.pyplot import *
import emcee

### First defining functions to be used for the mcmc fit

In [19]:
def log_likelihood(theta, X, B, Berr):
    IMFs = theta
    spec = getSpec(IMFs) # This is where the function will generate a model with given IMF and spit back out B_pred (break strength)
    Bpred = getBpred(spec)
    return -0.5 * (B - Bpred) **2 / Berr **2

def log_prior(theta):
    """Works so long as all the IMFs priors are from -10.0 to 10.0
    """
    IMFs = theta
    for IMF in IMFs:
        if -10.0 < IMF < 10.0:
            output = 0.0
        else:
            output = -np.inf
            break
    return output

def log_probability(theta, X, B, Berr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, X, B, Berr)

### Defining functions for grabbing the spectra and calculating the break strength

In [24]:
def getSpec(IMFs):

    return np.nan

def getBreakIndices(wspec, zred=None, **extras):

    a = 1.0 + zred
    
    blue, red = 3620, 4000
    
    all_indices = np.zeros(len(wspec), dtype=bool)
    blue_lower, blue_upper, red_lower, red_upper = False, False, False, False
    
    for i, wave in enumerate(wspec/a):
        if wave > blue and blue_lower==False:
            all_indices[i], blue_lower = True, True
        elif wave > blue+100 and blue_upper==False:
            all_indices[i-1], blue_upper = True, True
        elif wave > red and red_lower==False:
            all_indices[i], red_lower = True, True
        elif wave > red+100 and red_upper==False:
            all_indices[i-1], red_upper = True, True
    
    true_indices = [index for index, value in enumerate(all_indices) if value]
    return true_indices

def getBpred(spec):

    return np.nan

### Now defining functions relevant to generating the models specifically

In [107]:
def getModel(mass=None, zred=None, logzsol=None, tage=None, tau=None, dust2=None, imf_type=None, sfh_type=None, **extras):
    """Build a prospect.models.SpecModel object

    :param mass: (optional, default:None)
        If given, produce spectra for this mass. Otherwise the mass will
        be 1e10 solar masses.

    :param zred: (optional, default: None)
        If given, produce spectra and observed frame photometry appropriate
        for this redshift. Otherwise the redshift will be zero.

    :param logzsol: (optional, default: None)
        If given, fix the model metallicity (:math: `log(Z/Z_sun)`) to the given value.
        Otherwise the metallicity will be set to -0.5.
        
    :param tage: (optional, default: None)
        If given, produce spectra and model photometry appropriate for
        this galactic age. Otherwise the age will be set to 13. Gyrs.

    :param dust2: (optional, default: None)
        If given, produce spectra that are appropriate for provided dust
        attenuation. Otherwise attenuation will be set to 0.6.

    :returns model:
        An instance of prospect.models.SedModel
    """
    from prospect.models import SpecModel
    from prospect.models.templates import TemplateLibrary

    model_params = TemplateLibrary['parametric_sfh']
    model_params.update(TemplateLibrary['burst_sfh'])

    # Change `isfree` so that all parameters that will be kept track of are identified 
    # in the `model` object as `free_params`

    if zred is not None:
        model_params['zred']['init'] = zred

    if mass is not None:
        model_params['mass']['init'] = mass

    if logzsol is not None:
        model_params['logzsol']['init'] = logzsol

    if tage is not None:
        model_params['tage']['init'] = tage

    if fage_burst is not None:
        model_params['fage_burst']['init'] = fage_burst

    if fburst is not None:
        model_params['fburst']['init']

    if tau is not None:
        model_params['tau']['init'] = tau

    if dust2 is not None:
        model_params['dust2']['init'] = dust2

    if imf_type is not None:
        model_params['imf_type']['init'] = imf_type

    if sfh_type is not None:
        model_params['sfh']['init'] = sfh_type

    
    
    model = SpecModel(model_params)

    return model

In [176]:
model_params = TemplateLibrary['parametric_sfh']
model_params.update(TemplateLibrary['burst_sfh'])
model_params # Essentially a parametric sfh with a burst added on top with burst time controlled by `fage_burst`.

{'zred': {'N': 1,
  'isfree': False,
  'init': 0.1,
  'units': 'redshift',
  'prior': <class 'prospect.models.priors.TopHat'>(mini=0.0,maxi=4.0)},
 'mass': {'N': 1,
  'isfree': True,
  'init': 10000000000.0,
  'units': 'Solar masses formed',
  'prior': <class 'prospect.models.priors.LogUniform'>(mini=100000000.0,maxi=1000000000000.0)},
 'logzsol': {'N': 1,
  'isfree': True,
  'init': -0.5,
  'units': '$\\log (Z/Z_\\odot)$',
  'prior': <class 'prospect.models.priors.TopHat'>(mini=-2,maxi=0.19)},
 'dust2': {'N': 1,
  'isfree': True,
  'init': 0.6,
  'units': 'optical depth at 5500AA',
  'prior': <class 'prospect.models.priors.TopHat'>(mini=0.0,maxi=2.0)},
 'sfh': {'N': 1, 'isfree': False, 'init': 4, 'units': 'FSPS index'},
 'tage': {'N': 1,
  'isfree': True,
  'init': 1,
  'units': 'Gyr',
  'prior': <class 'prospect.models.priors.TopHat'>(mini=0.001,maxi=13.8)},
 'imf_type': {'N': 1, 'isfree': False, 'init': 2},
 'dust_type': {'N': 1, 'isfree': False, 'init': 0},
 'tau': {'N': 1,
  'isfr

-------------------


### Test cells

In [20]:
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Define a function to check for even numbers
def is_even(num):
    return num % 2 == 0

# Use filter to get only even numbers
even_numbers_iterator = filter(is_even, numbers)

# Convert the iterator to a list
even_numbers_list = list(even_numbers_iterator)
print(even_numbers_list)
# Output: [2, 4, 6, 8, 10]

[2, 4, 6, 8, 10]


In [22]:
from prospect.models.templates import TemplateLibrary

In [108]:
# Need to redefine all of the propspectFunctions for new use...

from prospectFunctions import getParams, getObs, getSps, getWave

imf_type = 2
grid_ranges, run_params = getParams(imf_type=imf_type, len_logz=100, len_dust=6, len_tage=100, dust=True)
a = 1.0 + run_params['zred']

logzs = grid_ranges['logzsol']
dusts = grid_ranges['dust2']
tages = grid_ranges['tage']

num_models = len(logzs)*len(dusts)*len(tages)

obs = getObs(**run_params)
sps = getSps(**run_params)
wspec, wphot = getWave(obs=obs, sps=sps, **run_params)

In [103]:
def getBreakIndices(wspec, zred=None, **extras):

    a = 1.0 + zred
    
    blue, red = 3620, 4000
    
    all_indices = np.zeros(len(wspec), dtype=bool)
    blue_lower, blue_upper, red_lower, red_upper = False, False, False, False
    
    for i, wave in enumerate(wspec/a):
        if wave > blue and blue_lower==False:
            all_indices[i], blue_lower = True, True
        elif wave > blue+100 and blue_upper==False:
            all_indices[i-1], blue_upper = True, True
        elif wave > red and red_lower==False:
            all_indices[i], red_lower = True, True
        elif wave > red+100 and red_upper==False:
            all_indices[i-1], red_upper = True, True
    
    true_indices = [index for index, value in enumerate(all_indices) if value]
    
    return true_indices

In [106]:
test = getBreakIndices(wspec, **run_params)
wspec[test]/a

array([3620.7322, 3719.7578, 4000.6309, 4099.6568])

In [69]:
test[test == True]
true_indices = [index for index, value in enumerate(test) if value]
print(true_indices, wspec[true_indices]/a)

[382, 492, 804, 914] [3620.7322 3719.7578 4000.6309 4099.6568]


In [71]:
wspec[381:383]/a, wspec[492:494]/a, wspec[803:805]/a, wspec[914:916]/a

(array([3619.832 , 3620.7322]),
 array([3719.7578, 3720.6582]),
 array([3999.7308, 4000.6309]),
 array([4099.6568, 4100.5569]))

### Getting a model param that matches the model type we're looking for

In [171]:
# TemplateLibrary.show_contents()

In [175]:
model_params = TemplateLibrary['parametric_sfh']
model_params.update(TemplateLibrary['burst_sfh'])
model_params # Essentially a parametric sfh with a burst added on top with burst time controlled by `fage_burst`.

{'zred': {'N': 1,
  'isfree': False,
  'init': 0.1,
  'units': 'redshift',
  'prior': <class 'prospect.models.priors.TopHat'>(mini=0.0,maxi=4.0)},
 'mass': {'N': 1,
  'isfree': True,
  'init': 10000000000.0,
  'units': 'Solar masses formed',
  'prior': <class 'prospect.models.priors.LogUniform'>(mini=100000000.0,maxi=1000000000000.0)},
 'logzsol': {'N': 1,
  'isfree': True,
  'init': -0.5,
  'units': '$\\log (Z/Z_\\odot)$',
  'prior': <class 'prospect.models.priors.TopHat'>(mini=-2,maxi=0.19)},
 'dust2': {'N': 1,
  'isfree': True,
  'init': 0.6,
  'units': 'optical depth at 5500AA',
  'prior': <class 'prospect.models.priors.TopHat'>(mini=0.0,maxi=2.0)},
 'sfh': {'N': 1, 'isfree': False, 'init': 4, 'units': 'FSPS index'},
 'tage': {'N': 1,
  'isfree': True,
  'init': 1,
  'units': 'Gyr',
  'prior': <class 'prospect.models.priors.TopHat'>(mini=0.001,maxi=13.8)},
 'imf_type': {'N': 1, 'isfree': False, 'init': 2},
 'dust_type': {'N': 1, 'isfree': False, 'init': 0},
 'tau': {'N': 1,
  'isfr